In [1]:
import numpy as np
import pandas as pd
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import time

# === Load and Preprocess Data ===
# df_adult = pd.read_csv('Datasets/adult/adult_test_int.csv')
# df_adult = df_adult.drop(columns=["Unnamed: 0"])
df_adult = pd.read_csv('Datasets/No_show_test_int.csv')
df_adult = df_adult.drop(columns=["Unnamed: 0"])

num_it = 50
num_layers = 3
#selected_features = ["age", "capital.gain", "capital.loss", "hours.per.week", "education"]
#y = df_adult["over50K"].values
selected_features = ["Age", "Gender", "Sms_Reminder", "AwaitingTime"]
y = df_adult["Show_Up"].values

In [2]:
scaler = StandardScaler()
df_adult[selected_features] = scaler.fit_transform(df_adult[selected_features])

X_quantum = df_adult[selected_features].values
y_quantum = y * 2 - 1  # Convert to {-1,1} for quantum classifier

X_train_q, X_test_q, y_train_q, y_test_q = train_test_split(
    X_quantum, y_quantum, test_size=0.10, random_state=42, stratify=y
)

# === Quantum Model Setup ===
num_qubits = len(selected_features)  # Update number of qubits to match feature count
dev = qml.device("default.qubit", wires=num_qubits)

# Benedetti-style feature encoding
def feature_encoding(x):
    for i in range(num_qubits):
        qml.RY(np.pi * x[i], wires=i)  # Use RY encoding instead of simple RZ
    for i in range(num_qubits - 1):
        qml.CZ(wires=[i, i + 1])  # Introduce entanglement

# Variational Layer from Benedetti et al.
def variational_layer(W):
    for i in range(num_qubits):
        qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)
    for i in range(num_qubits - 1):
        qml.CNOT(wires=[i, i + 1])  # Alternating CNOT layers
    qml.CNOT(wires=[num_qubits - 1, 0])

@qml.qnode(dev, interface="autograd")
def circuit(weights, x):
    feature_encoding(x)
    for W in weights:
        variational_layer(W)
    return qml.expval(qml.PauliZ(0))

def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

# === Training Setup ===
def cost(weights, bias, X, Y):
    predictions = qml.numpy.array([variational_classifier(weights, bias, x) for x in X])
    return qml.numpy.mean((qml.numpy.array(Y) - predictions) ** 2)

np.random.seed(0)
weights_init = qml.numpy.tensor(0.01 * np.random.randn(num_layers, num_qubits, 3), requires_grad=True)
bias_init = qml.numpy.tensor(0.0, requires_grad=True)

opt = qml.optimize.AdamOptimizer(0.05)
batch_size = min(48, len(X_train_q))

# === Train Quantum Model ===
weights, bias = weights_init, bias_init

for it in range(num_it):
    batch_index = np.random.choice(len(X_train_q), batch_size, replace=False)
    X_batch, Y_batch = X_train_q[batch_index].astype(np.float64), y_train_q[batch_index]

    weights, bias = opt.step(lambda w, b: cost(w, b, X_batch, Y_batch), weights, bias)

    predictions = np.array([qml.numpy.sign(variational_classifier(weights, bias, x)) for x in X_train_q])
    acc = np.mean(np.abs(y_train_q - predictions) < 1e-5)
    
    print(f"Iter: {it+1:5d} | Cost: {cost(weights, bias, X_train_q, y_train_q):0.7f} | Accuracy: {acc:0.7f}")

# Recompute predictions using the trained quantum model
predictions_q = np.array([
    float(qml.numpy.sign(variational_classifier(weights, bias, x))) 
    for x in X_test_q
])

# Compute evaluation metrics
accuracy = accuracy_score(y_test_q, predictions_q)
precision = precision_score(y_test_q, predictions_q, zero_division=1)
recall = recall_score(y_test_q, predictions_q, zero_division=1)
f1 = f1_score(y_test_q, predictions_q, average='macro')

# Print results
print("\nQuantum Model Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Iter:     1 | Cost: 1.5783359 | Accuracy: 0.3014444
Iter:     2 | Cost: 1.4705731 | Accuracy: 0.3014444
Iter:     3 | Cost: 1.3547277 | Accuracy: 0.3014444
Iter:     4 | Cost: 1.2377902 | Accuracy: 0.3650370
Iter:     5 | Cost: 1.1272282 | Accuracy: 0.4286296
Iter:     6 | Cost: 1.0310845 | Accuracy: 0.4609630
Iter:     7 | Cost: 0.9542702 | Accuracy: 0.5624444
Iter:     8 | Cost: 0.9048373 | Accuracy: 0.6292593
Iter:     9 | Cost: 0.8750897 | Accuracy: 0.6931852
Iter:    10 | Cost: 0.8643566 | Accuracy: 0.6985556
Iter:    11 | Cost: 0.8654881 | Accuracy: 0.6985556
Iter:    12 | Cost: 0.8737837 | Accuracy: 0.6985556
Iter:    13 | Cost: 0.8836415 | Accuracy: 0.6985556
Iter:    14 | Cost: 0.8887784 | Accuracy: 0.6985556
Iter:    15 | Cost: 0.8919498 | Accuracy: 0.6985556
Iter:    16 | Cost: 0.8931842 | Accuracy: 0.6985556
Iter:    17 | Cost: 0.8920793 | Accuracy: 0.6985556
Iter:    18 | Cost: 0.8879608 | Accuracy: 0.6985556
Iter:    19 | Cost: 0.8818256 | Accuracy: 0.6985556
Iter:    20 

In [3]:
# === Classical Model Setup ===

# Select same features for Classical Model
X_classical = df_adult[selected_features].values  # Matches Quantum Model

# Update Train/Test Split
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_classical, y, test_size=0.10, random_state=42, stratify=y
)

# Define Classical ANN Model
class ClassicalANN(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size=5):
        super().__init__()
        layers = [input_size] + [hidden_size] * (num_layers - 1) + [1]  # Dynamic layer sizes
        self.layers = nn.ModuleList([nn.Linear(layers[i], layers[i+1], dtype=torch.float64) for i in range(len(layers) - 1)])
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.to(torch.float64)
        for layer in self.layers[:-1]:
            x = torch.relu(layer(x))
        return self.sigmoid(self.layers[-1](x))

# Train Classical Model
input_size = len(selected_features)  # Automatically adjust input size
classical_model = ClassicalANN(input_size, num_layers)  # Uses num_layers for hidden depth
optimizer_classical = optim.Adam(classical_model.parameters(), lr=0.01)

def train_classical(model, optimizer, X_train, y_train, epochs=50):
    y_train = torch.tensor(y_train.tolist(), dtype=torch.float64).reshape(-1, 1)
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = model(torch.tensor(X_train.tolist(), dtype=torch.float64)).reshape(-1, 1)
        
        # Compute loss
        loss = nn.BCELoss()(y_pred, y_train)
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        y_pred_binary = (y_pred.detach().numpy().flatten() > 0.5).astype(int)
        acc = accuracy_score(y_train.numpy().flatten(), y_pred_binary)
        
        # Debug output
        print(f"Iter: {epoch+1:5d} | Cost: {loss.item():0.7f} | Accuracy: {acc:0.7f}")

# Train the Classical Model with Debugging Output
train_classical(classical_model, optimizer_classical, X_train_c, y_train_c, epochs=num_it)

# === Evaluate Classical Model ===
with torch.no_grad():
    X_test_c_numeric = np.array(X_test_c, dtype=np.float64)
    y_pred_classical = classical_model(torch.tensor(X_test_c_numeric, dtype=torch.float64)).reshape(-1, 1)
    y_pred_classical = (y_pred_classical.numpy().flatten() > 0.5).astype(int)

print("\nClassical Model Performance:")
print(f"Accuracy: {accuracy_score(y_test_c, y_pred_classical):.4f}")
print(f"Precision: {precision_score(y_test_c, y_pred_classical, zero_division=1):.4f}")
print(f"Recall: {recall_score(y_test_c, y_pred_classical, zero_division=1):.4f}")
print(f"F1 Score: {f1_score(y_test_c, y_pred_classical, average='macro'):.4f}")


Iter:     1 | Cost: 0.6483373 | Accuracy: 0.6985556
Iter:     2 | Cost: 0.6454453 | Accuracy: 0.6985556
Iter:     3 | Cost: 0.6429669 | Accuracy: 0.6985556
Iter:     4 | Cost: 0.6408171 | Accuracy: 0.6985556
Iter:     5 | Cost: 0.6389565 | Accuracy: 0.6985556
Iter:     6 | Cost: 0.6373933 | Accuracy: 0.6985556
Iter:     7 | Cost: 0.6359890 | Accuracy: 0.6985556
Iter:     8 | Cost: 0.6346891 | Accuracy: 0.6985556
Iter:     9 | Cost: 0.6334662 | Accuracy: 0.6985556
Iter:    10 | Cost: 0.6322850 | Accuracy: 0.6985556
Iter:    11 | Cost: 0.6311238 | Accuracy: 0.6985556
Iter:    12 | Cost: 0.6299668 | Accuracy: 0.6985556
Iter:    13 | Cost: 0.6288207 | Accuracy: 0.6985556
Iter:    14 | Cost: 0.6276883 | Accuracy: 0.6985556
Iter:    15 | Cost: 0.6264862 | Accuracy: 0.6985556
Iter:    16 | Cost: 0.6252083 | Accuracy: 0.6985556
Iter:    17 | Cost: 0.6239148 | Accuracy: 0.6985556
Iter:    18 | Cost: 0.6225739 | Accuracy: 0.6985556
Iter:    19 | Cost: 0.6212085 | Accuracy: 0.6985556
Iter:    20 